In [ ]:
#51. 特徴量抽出
#学習データ，検証データ，評価データから特徴量を抽出し，
#それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ．
#なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．
#記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

#方針１　ライブラリのインポート  Pandas：データフレームの操作、
#　　　　                        string、re：テキストデータの前処理、
#　　　　　　　　　　　　　　　　sklearn.feature_extractionモジュール：テキストや画像などの形式からなるデータセットから特徴抽出
#                                   TfidfVectorizer：TF-IDF(索引語頻度逆文書頻度)により特徴量
#　　　　　　　　　　　　　　　　　　　　　　　　　　(TF-IDF)=(TF（文書出現頻度）)* (IDF（逆文書頻度）)　　※逆文書頻度：単語のレア度
#　　　　　　　　　　　　　　　　　　　　　　　　　　IDF= log(総文書数/指定単語を含む文書数)
#                                   ※CountVectorizer：出現単語を数えて特徴量とする
#                                                    　問題点：出現数の多い単語のベクトルが極端に強くなる

#方針２　前処理を関数化する　prep()関数：テキストデータの句読点の削除、小文字化、数字の削除

#方針３　train.txt、valid.txt、test.txtからread_csv()でデータを読み込む

#方針４　データの結合と前処理: 3つのデータセットをconcat()で結合し、reset_index()結合後のDataFrameのインデックスを連続的に整理
#                              →TITLEカラムのデータにapplyで前処理関数を適用

#方針５　データを「トレーニング、検証データセット/テストデータセット」分割し、TfidfVectorizerでtf-idf特徴量の抽出

#方針６　統計量を計算し、その統計量をもとに特徴ベクトルをデータフレーム化→train.feature.txt、valid.feature.txt、test.feature.txtとして保存

#方針７　特徴ベクトル（トレーニングデータの最初の数行）を表示




#作業１ライブラリのインポート
import pandas as pd
import string
import re
from sklearn.feature_extraction.text import CountVectorizer #BoW
from sklearn.feature_extraction.text import TfidfVectorizer



#作業２：前処理の関数化
def prep(text):
    text = re.sub(f"[{string.punctuation}]", "", text) #string.punctuation ：!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~
    text = text.lower()  # 小文字化
    text = re.sub("[0-9]+", "", text)  # 1つ以上の数字を""で置換
    return text

#作業３：テキストファイル読み込み
header_name = ['TITLE', 'CATEGORY']

train = pd.read_csv('./train.txt', header=None, sep='\t', names=header_name)
valid = pd.read_csv('./valid.txt', header=None, sep='\t', names=header_name)
test = pd.read_csv('./test.txt', header=None, sep='\t', names=header_name)


#作業４：データの結合,前処理関数適用
df = pd.concat([train, valid, test], axis=0)    #concat([連結するDataFrame、Series(表1列) #カンマ区切り], axis=0 #縦方向)
df.reset_index(drop=True, inplace=True) #drop=True：元のインデックス列を削除(DataFrameのサイズ削減)
                                        #inplace=True:元のDataFrameを直接変更(デフォルトだと元のDataFrameを変更し、新しいDataFrameを返す）


df["TITLE"] = df["TITLE"].apply(prep)

#作業５：データ分割、特徴量抽出
train_valid_data = df[:len(train)+len(valid)] #トレーニングデータと検証データの合計行数までの行をtrain_valid_dに割り当て
test_data = df[len(train)+len(valid):]

vec_tfidf = TfidfVectorizer(min_df=10, ngram_range=(1, 2)) #min_df：dfの最小値を設定
                                                           #n_gram_range：使用するn_gramを1~2gramに設定

#作業６：統計量の正規化、データフレームに変換、それぞれファイルに保存

#fit_transform()：fitで統計量を計算し、その統計量に基いてtransformで()の中身を正規化
#TAの方のアドバイス：valid dataも省いて、トレーニングデータだけを学習する(fit_transform)べき！
train_valid_f = vec_tfidf.fit_transform(train_valid_data["TITLE"])
test_f = vec_tfidf.transform(test_data["TITLE"]) #1行前で計算した統計量をもとにtransform(テストデータを学習に用いてはいけない)


#toarray()メソッド：疎行列(Scipyのcsr_matrix形式)→密行列(Numpyのndarray形式)に変換
#                   (疎行列はメモリ効率が良いが、機械学習アルゴリズムやデータ解析の多くは密行列が前提)
#                     ※CSR形式について：https://qiita.com/iwasaki620/items/603220d9102e82d4438e

#get_feature_names_out()メソッド：ベクトライザが生成した特徴(単語、n-gram)のリストを返す
#                                 データフレームの各列がどの単語、n-gramを表すかを示すラベル付与
train_valid_vec = pd.DataFrame(train_valid_f.toarray(),
                               columns=vec_tfidf.get_feature_names_out()) #列：特徴量、行：タイトル

test_vec = pd.DataFrame(test_f.toarray(),
                        columns=vec_tfidf.get_feature_names_out())

train_vec = train_valid_vec[:len(train)]
valid_vec = train_valid_vec[len(train):]

train_vec.to_csv("./train.feature.txt", sep="\t", index=False)
valid_vec.to_csv("./valid.feature.txt", sep="\t", index=False)
test_vec.to_csv("./test.feature.txt", sep="\t", index=False)

#作業７：特徴ベクトルの表示
print(train_vec.head())

    aa  aaliyah  abbvie  abc     about  about the  about to  above  abramson  \
0  0.0      0.0     0.0  0.0  0.000000        0.0       0.0    0.0       0.0   
1  0.0      0.0     0.0  0.0  0.274054        0.0       0.0    0.0       0.0   
2  0.0      0.0     0.0  0.0  0.000000        0.0       0.0    0.0       0.0   
3  0.0      0.0     0.0  0.0  0.000000        0.0       0.0    0.0       0.0   
4  0.0      0.0     0.0  0.0  0.000000        0.0       0.0    0.0       0.0   

   abuse  ...  young  your  your mother   yr  yr high  yuan  zac  zac efron  \
0    0.0  ...    0.0   0.0          0.0  0.0      0.0   0.0  0.0        0.0   
1    0.0  ...    0.0   0.0          0.0  0.0      0.0   0.0  0.0        0.0   
2    0.0  ...    0.0   0.0          0.0  0.0      0.0   0.0  0.0        0.0   
3    0.0  ...    0.0   0.0          0.0  0.0      0.0   0.0  0.0        0.0   
4    0.0  ...    0.0   0.0          0.0  0.0      0.0   0.0  0.0        0.0   

   zendaya  zone  
0      0.0   0.0  
1     